In [0]:
OUTPUTPath = './output/'
!mkdir ./output

In [0]:
# Save model to your Drive
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
import pandas as pd
DATA_SET_DIR = './gdrive/My Drive/NLP/Final-Project/data/'
TRAINING_DATA_PATH = DATA_SET_DIR+'olid-training-clean.csv'
TEST_PATH_DICT = {}
TEST_A_DATA_PATH = DATA_SET_DIR+'testset-levela-clean.tsv'
TEST_B_DATA_PATH = DATA_SET_DIR+'testset-levelb-clean.tsv'
TEST_C_DATA_PATH = DATA_SET_DIR+'testset-levelc-clean.tsv'
TEST_PATH_DICT['subtask_a'] = TEST_A_DATA_PATH
TEST_PATH_DICT['subtask_b'] = TEST_B_DATA_PATH
TEST_PATH_DICT['subtask_c'] = TEST_C_DATA_PATH

data={}
data['train'] = {}
data['validation'] = {}
data['test'] = {}
task_list = ['subtask_a','subtask_b','subtask_c']
train = pd.read_csv(TRAINING_DATA_PATH,sep='\t', index_col='id')
for subtask in task_list:
    data['test'][subtask] = pd.read_csv(TEST_PATH_DICT[subtask],sep='\t', index_col='id')
train.head(3)

,tweet_exp,subtask_a,subtask_b,subtask_c
id,,,,
86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
90194,@USER @USER Go home you are drunk!!! @USER #MA...,OFF,TIN,IND
16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN


In [0]:
print(train.isna().any())
print('-'*10)
print(train.subtask_a.value_counts())
print(train.subtask_b.value_counts())
print(train.subtask_c.value_counts())

tweet_exp    False
subtask_a    False
subtask_b     True
subtask_c     True
dtype: bool
----------
NOT    8840
OFF    4400
Name: subtask_a, dtype: int64
TIN    3876
UNT     524
Name: subtask_b, dtype: int64
IND    2407
GRP    1074
OTH     395
Name: subtask_c, dtype: int64


In [0]:
import re, string
def _run_split_on_punc(text):
    """Splits punctuation on a piece of text."""
    chars = list(text)
    i = 0
    start_new_word = True
    output = []
    def _is_punctuation(char):
        # print('[%s]+' % re.escape(string.punctuation))
        if re.match(r'[\!\"\$\%\&\(\)\*\+\,\-\.\/\:\;\<\=\>\?\[\\\]\^_\`\{\|\}\~]+',char):
            return True
        else:
            return False
    while i < len(chars):
        char = chars[i]
        if _is_punctuation(char):
            output.append([char])
            start_new_word = True
        else:
            if start_new_word:
                output.append([])
            start_new_word = False
            output[-1].append(char)
        i += 1
    text_list = ["".join(x) for x in output]
    text = ' '.join(text_list)
    text = text.split(' ')
    return ' '.join(text)
print(_run_split_on_punc("@USER @USER @USER @USER LOL!!!   Throwing the BULLSHIT Flag on such nonsense!!  #PutUpOrShutUp"))

@USER @USER @USER @USER LOL ! ! !    Throwing the BULLSHIT Flag on such nonsense ! !   #PutUpOrShutUp


In [0]:
train.tweet_exp = train.tweet_exp.apply(_run_split_on_punc)
for subtask in task_list:
    data['test'][subtask].tweet_exp = data['test'][subtask].tweet_exp.apply(_run_split_on_punc)

In [0]:
classes_dict = {}
classes_dict['subtask_a'] = {'NOT' : 0 ,'OFF' : 1}
classes_dict['subtask_b'] = {'UNT' : 0 ,'TIN' : 1}
classes_dict['subtask_c'] = {'IND' : 0 ,'OTH' : 1, 'GRP' : 2}
for subtask in task_list:
    data['train'][subtask]=pd.DataFrame()
    for label in classes_dict[subtask]:
        data_label = train[train[subtask]==label]
        data['train'][subtask] = data['train'][subtask].append(train.loc[data_label.index])
    print(subtask,":")
    print(data['train'][subtask][subtask].value_counts())
    print(len(data['train'][subtask]))
    print('-'*10)

subtask_a :
NOT    8840
OFF    4400
Name: subtask_a, dtype: int64
13240
----------
subtask_b :
TIN    3876
UNT     524
Name: subtask_b, dtype: int64
4400
----------
subtask_c :
IND    2407
GRP    1074
OTH     395
Name: subtask_c, dtype: int64
3876
----------


In [0]:
import numpy as np
corpus_train = train.tweet_exp
corpus = pd.concat([train.tweet_exp, data['test']['subtask_a'].tweet_exp,data['test']['subtask_b'].tweet_exp,data['test']['subtask_c'].tweet_exp])
print(train.shape,corpus.shape)
print(data['test']['subtask_a'].shape,data['test']['subtask_b'].shape,data['test']['subtask_c'].shape)
pd.DataFrame(corpus.iloc[:5],columns=['tweet_exp'])

(13240, 4) (14553,)
(860, 1) (240, 1) (213, 1)


,tweet_exp
id,
86426,@USER She should ask a few native Americans wh...
90194,@USER @USER Go home you are drunk ! ! ! @USER...
16820,Amazon is investigating Chinese employees who ...
62688,"@USER Someone should'veTaken "" this piece of ..."
43605,@USER @USER Obama wanted liberals & amp ; il...


In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
MAX_NB_WORDS=10000
tokenizer = Tokenizer(nb_words=None,filters='')
tokenizer.fit_on_texts(corpus)
X = tokenizer.texts_to_sequences(corpus_train)

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [0]:
for seq in X[:3]:
    print(seq)
    print([tokenizer.index_word[idx] for idx in seq])
MAX_SEQUENCE_LENGTH = max([len(X[i]) for i in range(len(X))])
print('MAX_SEQUENCE_LENGTH:',MAX_SEQUENCE_LENGTH)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

[1, 19, 87, 374, 7, 468, 2899, 311, 39, 72, 138, 27, 24, 4, 2]
['@user', 'she', 'should', 'ask', 'a', 'few', 'native', 'americans', 'what', 'their', 'take', 'on', 'this', 'is', '.']
[1, 1, 102, 338, 6, 10, 1008, 9, 9, 9, 1, 56, 1053, 20]
['@user', '@user', 'go', 'home', 'you', 'are', 'drunk', '!', '!', '!', '@user', '#maga', '#trump2020', 'url']
[2900, 4, 7358, 2141, 2649, 51, 10, 1737, 5648, 1574, 5, 1738, 29, 141, 10257, 291, 21, 66, 2901, 17, 3, 2902, 5649, 2, 20, 10258, 56, 423, 4680, 478]
['amazon', 'is', 'investigating', 'chinese', 'employees', 'who', 'are', 'selling', 'internal', 'data', 'to', 'third', '-', 'party', 'sellers', 'looking', 'for', 'an', 'edge', 'in', 'the', 'competitive', 'marketplace', '.', 'url', '#amazon', '#maga', '#kag', '#china', '#tcot']
MAX_SEQUENCE_LENGTH: 113
Found 22043 unique tokens.


In [0]:
import keras
from keras.preprocessing.sequence import pad_sequences
def text2id(tokenizer,texts_pd,max_seq_length=MAX_SEQUENCE_LENGTH):
  x_training = tokenizer.texts_to_sequences(texts_pd)
  x_training = keras.preprocessing.sequence.pad_sequences(x_training,maxlen=max_seq_length)
  for seq in x_training:
      assert len(seq) == max_seq_length
  print("all sequence length are ",max_seq_length)
  return x_training
for subtask in task_list:
  data['train'][subtask+'_token_id'] = text2id(tokenizer, data['train'][subtask].tweet_exp)

all sequence length are  113
all sequence length are  113
all sequence length are  113


In [0]:
# 將分類標籤對應到剛定義的數字
for subtask in task_list:
  data['train'][subtask+'_y'] = data['train'][subtask][subtask].apply(lambda x: classes_dict[subtask][x])
  print(data['train'][subtask+'_y'].value_counts())
  data['train'][subtask+'_y'] = np.asarray(data['train'][subtask+'_y']).astype('float32')
  print(data['train'][subtask+'_y'][:5])

0    8840
1    4400
Name: subtask_a, dtype: int64
[0. 0. 0. 0. 0.]
1    3876
0     524
Name: subtask_b, dtype: int64
[0. 0. 0. 0. 0.]
0    2407
2    1074
1     395
Name: subtask_c, dtype: int64
[0. 0. 0. 0. 0.]


In [0]:
for key in data.keys():
  print(key)
  print('*'*10)
  for sub_key in data[key].keys():
    print(sub_key)

train
**********
subtask_a
subtask_b
subtask_c
subtask_a_token_id
subtask_b_token_id
subtask_c_token_id
subtask_a_y
subtask_b_y
subtask_c_y
validation
**********
test
**********
subtask_a
subtask_b
subtask_c


In [0]:
from sklearn.model_selection import train_test_split
NUM_LSTM_UNITS = 128
# y_train = keras.utils.to_categorical(y_train)

VALIDATION_RATIO = 0.1
# 小彩蛋
RANDOM_STATE = 9527
for subtask in task_list:
  if subtask == 'subtask_c':
    data['train'][subtask+'_y'] = keras.utils.to_categorical(data['train'][subtask+'_y'])
  data['train'][subtask+'_x_train'], data['validation'][subtask+'_x_val'], \
  data['train'][subtask+'_y_train'], data['validation'][subtask+'_y_val'] =train_test_split(data['train'][subtask+'_token_id'],data['train'][subtask+'_y'],test_size=VALIDATION_RATIO,random_state=RANDOM_STATE)
  print("Training Set",'----',subtask)
  print("-" * 10)
  print("x_train: ", data['train'][subtask+'_x_train'].shape)
  print("y_train : ", data['train'][subtask+'_y_train'].shape)

  print("-" * 10)
  print("x_val:   ", data['validation'][subtask+'_x_val'].shape)
  print("y_val :   ", data['validation'][subtask+'_y_val'].shape)
  print("#" * 10)

Training Set ---- subtask_a
----------
x_train:  (11916, 113)
y_train :  (11916,)
----------
x_val:    (1324, 113)
y_val :    (1324,)
##########
Training Set ---- subtask_b
----------
x_train:  (3960, 113)
y_train :  (3960,)
----------
x_val:    (440, 113)
y_val :    (440,)
##########
Training Set ---- subtask_c
----------
x_train:  (3488, 113)
y_train :  (3488, 3)
----------
x_val:    (388, 113)
y_val :    (388, 3)
##########


In [0]:
for key in data.keys():
  print(key,' ---------->')
  for sub_key in data[key].keys():
    print(sub_key)
  print('*'*10)

train  ---------->
subtask_a
subtask_b
subtask_c
subtask_a_token_id
subtask_b_token_id
subtask_c_token_id
subtask_a_y
subtask_b_y
subtask_c_y
subtask_a_x_train
subtask_a_y_train
subtask_b_x_train
subtask_b_y_train
subtask_c_x_train
subtask_c_y_train
**********
validation  ---------->
subtask_a_x_val
subtask_a_y_val
subtask_b_x_val
subtask_b_y_val
subtask_c_x_val
subtask_c_y_val
**********
test  ---------->
subtask_a
subtask_b
subtask_c
subtask_a_x_test
subtask_b_x_test
**********


In [0]:
GLOVE_DIR='./gdrive/My Drive/NLP/Final-Project/WordEmb'
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.twitter.27B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 1193514 word vectors.


In [0]:
EMBEDDING_DIM = 100
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
null_words = open('./output/null-word.txt', 'w', encoding='utf-8')
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        null_words.write(word + '\n')
print('###[EMB] Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

###[EMB] Null word embeddings: 5696


In [0]:
from keras.optimizers import Adam
import pickle
from keras.models import *
from keras.layers import *
from keras.callbacks import ModelCheckpoint

In [0]:
def CNN_binary_class(IsBinaryClass=True):
  input_data = Input(shape=(MAX_SEQUENCE_LENGTH,),dtype='int32')
  emb_x = Embedding(len(word_index) + 1,
                              EMBEDDING_DIM,
                              weights=[embedding_matrix],
                              input_length=MAX_SEQUENCE_LENGTH,
                              trainable=True)(input_data)
  conv1_x = Conv1D(64, 2, activation='relu')(emb_x)
  x_1 =GlobalMaxPooling1D()(conv1_x)
  conv2_x = Conv1D(64, 3, activation='relu')(emb_x)
  x_2 = GlobalMaxPooling1D()(conv2_x)
  conv3_x = Conv1D(64, 5, activation='relu')(emb_x)
  x_3 = GlobalMaxPooling1D()(conv3_x)

  merged = concatenate([x_1, x_2,x_3],axis=-1)
  merged=BatchNormalization()(merged)
  merged = Dense(units=300,kernel_initializer='random_normal')(merged)
  merged=PReLU()(merged)
  merged=BatchNormalization()(merged)
  if IsBinaryClass:
    predictions = Dense(1, activation='sigmoid', kernel_initializer='random_normal')(merged)
  else:
    predictions = Dense(3, activation='softmax', kernel_initializer='random_normal')(merged)

  model = Model(
      inputs=input_data,
      outputs=predictions)
  return model

In [0]:
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
class Metrics(Callback):
    def __init__(self, filepath,validation_data=(),IsBinaryClass=True):
        self.file_path = filepath
        self.X_val, self.y_val = validation_data
        if not IsBinaryClass:
          self.y_val = np.argmax(self.y_val,axis=1)
        self.IsBinaryClass_ = IsBinaryClass
    def on_train_begin(self, logs={}):
        self.val_f1s = []
        self.best_val_f1 = 0
        self.val_recalls = []
        self.val_precisions = []

    def on_epoch_end(self, epoch, logs={}):
        if self.IsBinaryClass_:
          val_predict = (np.asarray(self.model.predict(self.X_val))).round().astype('int').reshape(-1)
        else:
          val_predict = np.argmax(self.model.predict(self.X_val),axis=1)
        val_targ = self.y_val
        _val_f1 = f1_score(val_targ, val_predict,average='macro')
#         _val_recall = recall_score(val_targ, val_predict)
#         _val_precision = precision_score(val_targ, val_predict)
        self.val_f1s.append(_val_f1)
#         self.val_recalls.append(_val_recall)
#         self.val_precisions.append(_val_precision)
#         print(' — val_f1: %f — val_precision: %f — val_recall %f' %(_val_f1, _val_precision, _val_recall))
        print(' — val_f1: %f' %(_val_f1))
        print("max f1; ",max(self.val_f1s))
        if _val_f1 > self.best_val_f1:
            self.model.save(self.file_path, overwrite=True)
            self.best_val_f1 = _val_f1
            print("best f1: {}".format(self.best_val_f1))
        else:
            print("val f1: {}, but not the best f1".format(_val_f1))
        return

TARGET_TASK = 'subtask_a'  ----->  train subtask a

In [0]:
TARGET_TASK = 'subtask_c'
# data['validation'][TARGET_TASK+'_y_val'] = np.argmax(data['validation'][TARGET_TASK+'_y_val'],axis=1)
if TARGET_TASK == 'subtask_c':
  Is_Binary_Class = False
MODEL_SAVE_PATH='./output/CNN_Train_'+TARGET_TASK+'.hdf5' 
metrics = Metrics(MODEL_SAVE_PATH,
                  validation_data=(data['validation'][TARGET_TASK+'_x_val'],data['validation'][TARGET_TASK+'_y_val']),
                  IsBinaryClass=Is_Binary_Class)

model=CNN_binary_class(IsBinaryClass=Is_Binary_Class)
model.summary()
model.compile(loss='binary_crossentropy',
          optimizer= "adam",
          metrics=['accuracy'])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, 113)          0                                            
__________________________________________________________________________________________________
embedding_13 (Embedding)        (None, 113, 100)     2204400     input_13[0][0]                   
__________________________________________________________________________________________________
conv1d_37 (Conv1D)              (None, 112, 64)      12864       embedding_13[0][0]               
__________________________________________________________________________________________________
conv1d_38 (Conv1D)              (None, 111, 64)      19264       embedding_13[0][0]               
__________________________________________________________________________________________________
conv1d_39 

In [0]:
BATCH_SIZE = 512
NUM_EPOCHS = 50
history = model.fit(
    x=data['train'][TARGET_TASK+'_x_train'],
    y=data['train'][TARGET_TASK+'_y_train'],
    batch_size=BATCH_SIZE,
    epochs=NUM_EPOCHS,
    callbacks=[metrics],
    # 每個 epoch 完後計算驗證資料集
    # 上的 Loss 以及準確度
    validation_data=(data['validation'][TARGET_TASK+'_x_val'],data['validation'][TARGET_TASK+'_y_val']),
    # 每個 epoch 隨機調整訓練資料集
    # 裡頭的數據以讓訓練過程更穩定
    shuffle=True,
)

Train on 3488 samples, validate on 388 samples
Epoch 1/50
3488/3488 [==============================] - 3s 971us/step - loss: 0.6663 - acc: 0.6706 - val_loss: 0.5510 - val_acc: 0.7337
 — val_f1: 0.430729
max f1;  0.43072931146325644
best f1: 0.43072931146325644
Epoch 2/50
3488/3488 [==============================] - 0s 59us/step - loss: 0.4628 - acc: 0.8063 - val_loss: 0.5689 - val_acc: 0.7363
 — val_f1: 0.417058
max f1;  0.43072931146325644
val f1: 0.41705839075811957, but not the best f1
Epoch 3/50
3488/3488 [==============================] - 0s 44us/step - loss: 0.3705 - acc: 0.8609 - val_loss: 0.5015 - val_acc: 0.7689
 — val_f1: 0.504536
max f1;  0.5045361468551887
best f1: 0.5045361468551887
Epoch 4/50
3488/3488 [==============================] - 0s 40us/step - loss: 0.2951 - acc: 0.9099 - val_loss: 0.4933 - val_acc: 0.7792
 — val_f1: 0.463305
max f1;  0.5045361468551887
val f1: 0.46330519520323515, but not the best f1
Epoch 5/50
3488/3488 [==============================] - 0s 37us

In [0]:
file_his = open('./output/CNNV5_train_hist_'+TARGET_TASK+'.pickle', 'wb')
pickle.dump(history.history, file_his)
file_his.close()

In [0]:
data['test'][TARGET_TASK+'_x_test'] = text2id(tokenizer, data['test'][TARGET_TASK].tweet_exp)

all sequence length are  113


In [0]:
from keras.models import load_model
if os.path.exists(MODEL_SAVE_PATH):
      model=load_model(MODEL_SAVE_PATH)
    # 若成功加载前面保存的参数，输出下列信息
      print("checkpoint_loaded")
predictions = model.predict(data['test'][TARGET_TASK+'_x_test'])

checkpoint_loaded


In [0]:
predictions = np.argmax(predictions,axis=1)
predictions[:5]

array([2, 0, 0, 0, 0])

In [0]:
#binary classification
predictions=predictions.round().astype('int').reshape(-1)
predictions[:5]

array([1, 0, 1, 1, 0])

In [0]:
TEST_A_LABEL_PATH = DATA_SET_DIR+'labels-levela.csv'
TEST_B_LABEL_PATH = DATA_SET_DIR+'labels-levelb.csv'
TEST_C_LABEL_PATH = DATA_SET_DIR+'labels-levelc.csv'
TEST_LABEL_PATH_DICT = {}
TEST_LABEL_PATH_DICT['subtask_a'] = TEST_A_LABEL_PATH
TEST_LABEL_PATH_DICT['subtask_b'] = TEST_B_LABEL_PATH
TEST_LABEL_PATH_DICT['subtask_c'] = TEST_C_LABEL_PATH

true_label = pd.read_csv(TEST_LABEL_PATH_DICT[TARGET_TASK],sep=',',header=None)
true_label.head(3)

,0,1
0,15923,OTH
1,60133,GRP
2,83681,IND


In [0]:
y_test = true_label[1].apply(lambda x: classes_dict[TARGET_TASK][x])
y_test = np.asarray(y_test).astype('int')
y_test[:5]

array([1, 2, 0, 1, 0])

In [0]:
testing_f1 = f1_score(y_test, predictions,average='macro')
print('testing data f1-score:',testing_f1)

testing data f1-score: 0.48019058963668854


In [0]:
!zip cnn_outpu_combTask_3 output/*

  adding: output/CNN_noTrain.hdf5 (deflated 44%)
  adding: output/CNN_Train.hdf5 (deflated 17%)
  adding: output/CNN_Train_subtask_b.hdf5 (deflated 44%)
  adding: output/CNN_Train_subtask_c.hdf5 (deflated 47%)
  adding: output/CNNV5_train_hist_subtask_b.pickle (deflated 54%)
  adding: output/CNNV5_train_hist_subtask_c.pickle (deflated 56%)
  adding: output/null-word.txt (deflated 51%)


In [0]:
!mv cnn_outpu_combTask_3.zip './gdrive/My Drive/NLP/Final-Project/output/'